In [1]:
import numpy as np
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D 
from keras.layers.convolutional import MaxPooling2D 
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
K.set_image_dim_ordering('th')
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [7]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), 
                     input_shape=(3, 150, 150), 
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(12, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model

In [8]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rotation_range=40, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   rescale=1./255, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True, 
                                   fill_mode='nearest')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'CAX_Superhero_Train/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'CAX_Superhero_Train/test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 4346 images belonging to 12 classes.
Found 1087 images belonging to 12 classes.


In [9]:
early_stopping_monitor = EarlyStopping(monitor='val_acc', patience=2, verbose=1)
# build the model
model = larger_model()

model.fit_generator(train_generator, 
                    steps_per_epoch=4346//batch_size, 
                    epochs=50, 
                    validation_data=validation_generator, 
                    validation_steps=1087//batch_size, 
                    callbacks=[early_stopping_monitor], 
                    verbose=1)

Epoch 1/50
271/271 [==============================] - 564s 2s/step - loss: 2.3249 - acc: 0.1987 - val_loss: 2.2252 - val_acc: 0.2631
Epoch 2/50
271/271 [==============================] - 561s 2s/step - loss: 2.1700 - acc: 0.2533 - val_loss: 2.0314 - val_acc: 0.2873
Epoch 3/50
271/271 [==============================] - 559s 2s/step - loss: 2.0540 - acc: 0.2960 - val_loss: 1.9539 - val_acc: 0.3340
Epoch 4/50
271/271 [==============================] - 559s 2s/step - loss: 1.9917 - acc: 0.3159 - val_loss: 1.8428 - val_acc: 0.3582
Epoch 5/50
271/271 [==============================] - 559s 2s/step - loss: 1.9385 - acc: 0.3512 - val_loss: 1.7953 - val_acc: 0.4244
Epoch 6/50
271/271 [==============================] - 559s 2s/step - loss: 1.8538 - acc: 0.3794 - val_loss: 1.7477 - val_acc: 0.4216
Epoch 7/50
271/271 [==============================] - 559s 2s/step - loss: 1.8500 - acc: 0.3887 - val_loss: 1.6958 - val_acc: 0.4347
Epoch 8/50
271/271 [==============================] - 559s 2s/step - 

In [10]:
model.save_weights('first_try.h5')

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

generator = datagen.flow_from_directory(
        'CAX_Superhero_Test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)  # keep data in same order as labels

In [ ]:
probabilities = model.predict_generator(generator, verbose=0)

In [ ]:
# y_prob = model.predict(x) 
y_classes = probabilities.argmax(axis=-1)

In [ ]:
submission = pd.DataFrame(data=np.column_stack((test_temp.index.values+1,pred_class)), 
                          columns=['ImageId', 'Label'])